##Project name - Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies

##Team Id - PNT2022TMID39097

##Model Building

##For body

##1.Importing The Model Building Libraries

In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


##2.Loading The Model


In [2]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/ibm project files/Car damage/body/training'
valid_path = '/content/drive/MyDrive/ibm project files/Car damage/body/validation'

In [3]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


##3.Adding Flatten Layer

In [4]:
for layer in vgg16.layers:
    layer.trainable = False

In [5]:
folders = glob('/content/drive/MyDrive/ibm project files/Car damage/body/training/*')

In [6]:
folders

['/content/drive/MyDrive/ibm project files/Car damage/body/training/02-side',
 '/content/drive/MyDrive/ibm project files/Car damage/body/training/00-front',
 '/content/drive/MyDrive/ibm project files/Car damage/body/training/01-rear']

In [7]:
x = Flatten()(vgg16.output)

In [8]:
len(folders)

3

##4.Adding Output Layer

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

##5.Creating A model Object

In [10]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

##6.Configure The Learning Process

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

##7.Train The Model

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
trainPath = '/content/drive/MyDrive/ibm project files/Car damage/body/training'
testPath = '/content/drive/MyDrive/ibm project files/Car damage/body/validation'

In [15]:
training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [16]:
training_set.class_indices

{'00-front': 0, '01-rear': 1, '02-side': 2}

In [17]:
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
97/97 [==============================] - 650s 7s/step - loss: 1.2234 - accuracy: 0.5284 - val_loss: 1.0827 - val_accuracy: 0.6118
Epoch 2/25
97/97 [==============================] - 656s 7s/step - loss: 0.6422 - accuracy: 0.7678 - val_loss: 1.8828 - val_accuracy: 0.5647
Epoch 3/25
97/97 [==============================] - 655s 7s/step - loss: 0.6094 - accuracy: 0.7843 - val_loss: 0.9752 - val_accuracy: 0.6529
Epoch 4/25
97/97 [==============================] - 660s 7s/step - loss: 0.4152 - accuracy: 0.8338 - val_loss: 1.1492 - val_accuracy: 0.6294
Epoch 5/25
97/97 [==============================] - 651s 7s/step - loss: 0.2969 - accuracy: 0.8854 - val_loss: 0.8088 - val_accuracy: 0.6588
Epoch 6/25
97/97 [==============================] - 650s 7s/step - loss: 0.2217 - accuracy: 0.9205 - val_loss: 1.0630 - val_accuracy: 0.6588
Epoch 7/25
97/97 [==============================] - 649s 7s/step - loss: 0.1782 - accuracy: 0.9412 - val_loss: 0.9224 - val_accuracy: 0.6588
Epoch 8/25
97

##8.Save The Model

In [21]:
from tensorflow.keras.models import load_model


In [22]:
model.save('/content/damage vehicle/Model/body.h5')

##9.Test The Model

In [23]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [25]:
model = load_model('/content/damage vehicle/Model/body.h5')

In [26]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [27]:
import numpy as np

In [28]:
data = "/content/drive/MyDrive/ibm project files/Car damage/body/training/00-front/0002.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 682ms/step
front


##Model Building

##For Level

##1. Importing The Model Building Libraries

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

##2. Loading The Model

In [30]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/ibm project files/Car damage/level/training'
valid_path = '/content/drive/MyDrive/ibm project files/Car damage/level/validation'

In [31]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

##3. Adding Flatten Layer

In [32]:
for layer in vgg16.layers:
    layer.trainable = False

In [33]:
folders = glob('/content/drive/MyDrive/ibm project files/Car damage/level/validation/*')

In [34]:
folders

['/content/drive/MyDrive/ibm project files/Car damage/level/validation/02-moderate',
 '/content/drive/MyDrive/ibm project files/Car damage/level/validation/01-minor',
 '/content/drive/MyDrive/ibm project files/Car damage/level/validation/03-severe']

In [35]:
x = Flatten()(vgg16.output)

In [36]:
len(folders)

3

##4. Adding Output Layer

In [37]:
prediction = Dense(len(folders), activation='softmax')(x)

##5. Creating A Model Object

In [38]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [39]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

##6. Configure The Learning Process

In [40]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

##7. Train The Model

In [42]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [43]:
trainPath = '/content/drive/MyDrive/ibm project files/Car damage/level/training'
testPath = '/content/drive/MyDrive/ibm project files/Car damage/level/validation'

In [44]:
training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [45]:
training_set.class_indices

{'01-minor': 0, '02-moderate': 1, '03-severe': 2}

In [47]:
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
97/97 [==============================] - 634s 7s/step - loss: 1.2682 - accuracy: 0.5366 - val_loss: 0.8523 - val_accuracy: 0.6059
Epoch 2/25
97/97 [==============================] - 627s 6s/step - loss: 0.7518 - accuracy: 0.7100 - val_loss: 0.8500 - val_accuracy: 0.6412
Epoch 3/25
97/97 [==============================] - 628s 6s/step - loss: 0.5602 - accuracy: 0.7812 - val_loss: 1.2610 - val_accuracy: 0.5588
Epoch 4/25
97/97 [==============================] - 627s 6s/step - loss: 0.4048 - accuracy: 0.8400 - val_loss: 0.9745 - val_accuracy: 0.6471
Epoch 5/25
97/97 [==============================] - 627s 6s/step - loss: 0.3305 - accuracy: 0.8834 - val_loss: 1.1164 - val_accuracy: 0.6176
Epoch 6/25
97/97 [==============================] - 628s 6s/step - loss: 0.2707 - accuracy: 0.8968 - val_loss: 1.2373 - val_accuracy: 0.5824
Epoch 7/25
97/97 [==============================] - 626s 6s/step - loss: 0.2523 - accuracy: 0.9061 - val_loss: 1.1922 - val_accuracy: 0.6000
Epoch 8/25
97

##8. Save The Model

In [48]:
from tensorflow.keras.models import load_model



In [49]:
model.save('/content/damage vehicle/Model/level.h5')

##9. Test The Model

In [50]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [51]:
model.save('/content/damage vehicle/Model/level.h5')

In [52]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["minor","moderate","severe"]
  preds = label[np.argmax(prediction)]
  return preds

In [53]:
import numpy as np

In [54]:
data = "/content/drive/MyDrive/ibm project files/Car damage/level/validation/01-minor/0005.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 706ms/step
minor
